In [1]:
# Customer ranking using POS data 
#   for Marketing Campaigns
import csv
import pandas as pd
import random as r
import sklearn


In [6]:
#---------------
# Compute Customer statistics for analysis
def customer_stats(df_pos):
    df = df_pos
    #Non negative filter
    df_nn = df[df["UnitPrice"]>0]

    # Group
    v_lineval=df_nn["Quantity"].astype(float) * df_nn["UnitPrice"].astype(float)
    df_nn=df_nn.assign(LineValue=v_lineval)

    #Non negative filter
    df_nn = df_nn[df_nn["LineValue"]>0]

    # Group the data frame by month and item and extract a number of stats from each group
    grouped_df = df_nn.groupby('CustomerID', as_index=False).agg({'LineValue': [min, max, sum],      # find the min, max, and sum of the duration column
                    'InvoiceNo': ["count"], # find the number of network type entries
                    'InvoiceDate': [min, max, 'first', 'nunique']})    # get the min, first, and number of unique dates per group

    grouped_df.columns = ["".join(x) for x in grouped_df.columns.ravel()]
    grouped_df.columns
    grouped_df.head()
    return(grouped_df)

#---------------
# Normalize functions
def fxnorm(x, mina, maxa):
    x_norm = (((x-float(mina))/(float(maxa)-float(mina)))*100)
    return x_norm

def flnorm(l, minx, maxx):
    l_norm = [fxnorm(i,minx, maxx) for i in l]
    return l_norm
#---------------
# Compute Customer Normals
def customer_normals(df_custstats):
    # Normalize Customer stat 1
    custvals=df_custstats.LineValuesum
    v_ValsNorm=flnorm(custvals, min(custvals), max(custvals))
    df_custstats=df_custstats.assign(LineValuesumNorm=v_ValsNorm)
    
    # Normalize Customer stat 2
    custvals=df_custstats.InvoiceNocount
    v_ValsNorm=flnorm(custvals, min(custvals), max(custvals))
    df_custstats=df_custstats.assign(InvoiceNocountNorm=v_ValsNorm)
    
    # Check Dataset
    df_custstats.columns
    df_custstats.head()
    
    df_analyze = pd.DataFrame({'LineValuesumNorm':df_custstats['LineValuesumNorm'].values,
                               'InvoiceNocountNorm':df_custstats['InvoiceNocountNorm'].values})
    return(df_analyze)
#---------------

# Customer ranking
def cust_ranks(df_analyze, df_custstats):
    from sklearn.cluster import KMeans
    # Convert DataFrame to matrix
    mat = df_analyze.as_matrix()
    # Using sklearn
    km = sklearn.cluster.KMeans(n_clusters=5)
    km.fit(mat)
    # Get cluster assignment labels
    labels = km.labels_
    # Format results as a DataFrame
    results = pd.DataFrame([df_analyze.index,labels]).T
    results
    results[0]
    
    df_analyze.head()
    
    # Get the Customer groups
    # Join the Groups
    df_CustGroups = pd.DataFrame({'CustomerGroup':results[1], 'CustomerID':df_custstats['CustomerID'].values})
    return(df_CustGroups)


In [2]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_ec7832a631cf488c8a552db7884026b6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9lpMGaOqG9N_xYlDNyNKYxVNjfKoxX5iTFFW-5bZ1NgA',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ec7832a631cf488c8a552db7884026b6.get_object(Bucket='campaignmanagementded83c5c44d440ebba56858d64bcdce0',Key='Online Retail Sample.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_pos = pd.read_csv(body)
df_pos.head()



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26:00,3.39,17850.0,United Kingdom


In [7]:
# ------------------
# Get the Customer by groups
import csv
import pandas as pd
import random as r
import sklearn
from sklearn.cluster import KMeans

#file_obj = "C:/Prabu/Cognitive/DevJourney_Campaign/Online Retail.csv"
#df = pd.read_csv(file_obj, header='infer', delimiter=',', skip_blank_lines=True, parse_dates=True, dtype=str)
#df[0:4]

df_custstats = customer_stats(df_pos)
df_custstats.head()

df_custanalyze = customer_normals(df_custstats)
df_custanalyze.head()

df_custRanked = cust_ranks(df_custanalyze, df_custstats)
df_custRanked.head()


,CustomerGroup,CustomerID
0,2,12431.0
1,1,12433.0
2,2,12583.0
3,2,12662.0
4,0,12748.0


In [9]:
# Decode results for Campaigns
results = df_custRanked
ranks = sorted(results.CustomerGroup.unique())
for i in ranks:
    print (i)
    print (results[results.CustomerGroup==i])


0
    CustomerGroup  CustomerID
4               0     12748.0
5               0     12791.0
13              0     13255.0
17              0     13576.0
18              0     13694.0
19              0     13705.0
20              0     13747.0
21              0     13748.0
25              0     14001.0
26              0     14045.0
29              0     14237.0
31              0     14496.0
40              0     15100.0
42              0     15235.0
43              0     15291.0
45              0     15350.0
47              0     15513.0
54              0     16048.0
62              0     16552.0
64              0     16835.0
65              0     16928.0
66              0     16955.0
67              0     17025.0
69              0     17181.0
72              0     17420.0
73              0     17460.0
75              0     17572.0
76              0     17643.0
78              0     17760.0
79              0     17809.0
82              0     17873.0
87              0     17924.0
88      